# Creating a Character Table from Genetic Strings

## Problem

A collection of strings is characterizable if there are at most two possible choices for the symbol at each position of the strings.

Given: A collection of at most 100 characterizable DNA strings, each of length at most 300 bp.

Return: A character table for which each nontrivial character encodes the symbol choice at a single position of the strings. (Note: the choice of assigning '1' and '0' to the two states of each SNP in the strings is arbitrary.)

## Phylogeny from Genetic Characters

In “Creating a Character Table”, we introduced the character table as a way of representing a number of characters simultaneously. In that problem, we found a character table representing an unrooted binary tree on a collection of taxa.

Of course, in practice, the problem operates in reverse. We first need to generate a character table before we can model a phylogeny on this table. In modern genetics, a reliable way to obtain a large number of characters is by using SNPs. As mentioned in “Counting Subsets”, for a given SNP, we can divide taxa into two sets depending on which of two bases is present at the nucleotide, thus defining the split of a character.


##Note

Recall that the character table does **not** encode **trivial** characters.


Origin: https://rosalind.info/problems/cstr/

## Sample Dataset
ATGCTACC
CGTTTACC
ATTCGACC
AGTCTCCC
CGTCTATC

## Sample Output
10110
10100



In [9]:
path = r"C:\Users\Hi\Downloads\rosalind_cstr.txt"
with open(path,'r') as file:
    rf = file.readlines()

In [10]:
import numpy as np

input = '''ATGCTACC
CGTTTACC
ATTCGACC
AGTCTCCC
CGTCTATC'''
input5 ='''...''' 
li = input.split('\n')

li = [i.replace('\n','') for i in rf]

print(li) 
#print(np.array(li).transpose())
nr = np.array(li).size
nc = len(li[0])
print(nr,nc)

['...']
86 299


In [11]:
res = []
for c in range(nc):
    temp = []
    for r in range(nr):
        if li[r][c] == li[0][c]:
            temp.append(1)
        else:
            temp.append(0)
    if temp.count(1)>1 and temp.count(0)>1:
        res.append(''.join(list(map(str,temp))))

In [12]:
[print(ri) for ri in set(res)]

11111111111111111111111111111111111111111111111111011011111111111111111111111111110110
11101111111111111111111101111111111111111111111111111111111111111111111111111111111111
11111111111111111111111111111111111110111111111111111111111111110111111111111111111111
11111111111111111111111111111111111111111111111111011111111111111111111111111111110111
10000010010100000000000000001100001000000100000000010000000100000000000101000010100100
11111011111100111111111111111111111111111101001111111111011111001011101111101111101111
11111011111111111111111111111111111111111111111111111111111111011111111111111111111111
11111111111111111110111111111100111111111111111111111110111111111101111111111111111111
11111111111111111111111111111111111111110111111101111111111111111111111111111111111111
11000010010100000000001000111100001010000100000000011000000100000000000101000110100110
11110110111111110011111111111111111011101111111111111111111111111111111111111111111111
1111111111111111111110111111111111111101111

[None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None]

In [13]:
import sys
sys.stdout.write('\n'.join(set(res)))

11111111111111111111111111111111111111111111111111011011111111111111111111111111110110
11101111111111111111111101111111111111111111111111111111111111111111111111111111111111
11111111111111111111111111111111111110111111111111111111111111110111111111111111111111
11111111111111111111111111111111111111111111111111011111111111111111111111111111110111
10000010010100000000000000001100001000000100000000010000000100000000000101000010100100
11111011111100111111111111111111111111111101001111111111011111001011101111101111101111
11111011111111111111111111111111111111111111111111111111111111011111111111111111111111
11111111111111111110111111111100111111111111111111111110111111111101111111111111111111
11111111111111111111111111111111111111110111111101111111111111111111111111111111111111
11000010010100000000001000111100001010000100000000011000000100000000000101000110100110
11110110111111110011111111111111111011101111111111111111111111111111111111111111111111
1111111111111111111110111111111111111101111

7220

In [15]:
def get_character_table(dna_strings):
    str_collection = []
    positions = [''.join(map(str, map(int, map(s[0].__eq__, s))))
                 for s in map(''.join, zip(*dna_strings))]

    for k in set(positions):
        if sum(map(int, k)) not in (0, 1, len(k), len(k) - 1):
            str_collection.append(k)

    return str_collection


with open(path) as f:
    dna_strings = map(str.strip, f.readlines())
    D = list(dna_strings)
    #RE = get_character_table(dna_strings)
print('\n'.join(str(x) for x in get_character_table(li)))
 

11111111111111111111111111111111111111111111111111011011111111111111111111111111110110
11101111111111111111111101111111111111111111111111111111111111111111111111111111111111
11111111111111111111111111111111111110111111111111111111111111110111111111111111111111
11111111111111111111111111111111111111111111111111011111111111111111111111111111110111
10000010010100000000000000001100001000000100000000010000000100000000000101000010100100
11111011111100111111111111111111111111111101001111111111011111001011101111101111101111
11111011111111111111111111111111111111111111111111111111111111011111111111111111111111
11111111111111111110111111111100111111111111111111111110111111111101111111111111111111
11111111111111111111111111111111111111110111111101111111111111111111111111111111111111
11000010010100000000001000111100001010000100000000011000000100000000000101000110100110
11110110111111110011111111111111111011101111111111111111111111111111111111111111111111
1111111111111111111110111111111111111101111

In [6]:
print(li==D)


True
